# **Set**
set is a collection of distinct elements with no order. Sets support efficient operations like union, intersection, and membership checking.
   - **Applications:** Handling duplicates, fast membership testing, database systems.

In [ ]:
# init
from .find_keyboard_row import *

## Find words that can be typed using letters from only one row of an American keyboard

In [ ]:
"""
Given a List of words, return the words that can be typed using letters of
alphabet on only one row's of American keyboard.

For example:
Input: ["Hello", "Alaska", "Dad", "Peace"]
Output: ["Alaska", "Dad"]

"""

In [ ]:
# Reference: https://leetcode.com/problems/keyboard-row/description/

def find_keyboard_row(words):
    """
    :type words: List[str]
    :rtype: List[str]
    """
    keyboard = [
        set('qwertyuiop'),
        set('asdfghjkl'),
        set('zxcvbnm'),
    ]
    result = []
    for word in words:
        for key in keyboard:
            if set(word.lower()).issubset(key):
                result.append(word)
    return result


## Randomized Set data structure supporting insert, remove, and random element retrieval operations

In [ ]:
#! /usr/bin/env python3

"""
Design a data structure that supports all following operations
in average O(1) time.

insert(val): Inserts an item val to the set if not already present.
remove(val): Removes an item val from the set if present.
random_element: Returns a random element from current set of elements.
           Each element must have the same probability of being returned.
"""


In [ ]:
import random

In [ ]:
class RandomizedSet():
    """
    idea: shoot
    """

    def __init__(self):
        self.elements = []
        self.index_map = {}  # element -> index

    def insert(self, new_one):
        if new_one in self.index_map:
            return
        self.index_map[new_one] = len(self.elements)
        self.elements.append(new_one)

    def remove(self, old_one):
        if not old_one in self.index_map:
            return
        index = self.index_map[old_one]
        last = self.elements.pop()
        self.index_map.pop(old_one)
        if index == len(self.elements):
            return
        self.elements[index] = last
        self.index_map[last] = index

    def random_element(self):
        return random.choice(self.elements)


def __test():
    rset = RandomizedSet()
    ground_truth = set()
    n = 64

    for i in range(n):
        rset.insert(i)
        ground_truth.add(i)

    # Remove a half
    for i in random.sample(range(n), n // 2):
        rset.remove(i)
        ground_truth.remove(i)

    print(len(ground_truth), len(rset.elements), len(rset.index_map))
    for i in ground_truth:
        assert(i == rset.elements[rset.index_map[i]])

    for i in range(n):
        print(rset.random_element(), end=' ')
    print()


if __name__ == "__main__":
    __test()

## Set Cover Problem - Optimal and Greedy Algorithms

#### Calculate the powerset of any iterable.

#### For a range of integers up to the length of the given list, make all possible combinations and chain them together as one object.

    

In [ ]:
from itertools import chain, combinations

In [ ]:
"""
Universe *U* of n elements
Collection of subsets of U:
    S = S1,S2...,Sm
    Where every substet Si has an associated cost.

Find a minimum cost subcollection of S that covers all elements of U

Example:
    U = {1,2,3,4,5}
    S = {S1,S2,S3}

    S1 = {4,1,3},    Cost(S1) = 5
    S2 = {2,5},      Cost(S2) = 10
    S3 = {1,4,3,2},  Cost(S3) = 3

    Output:
        Set cover = {S2, S3}
        Min Cost = 13
"""

In [ ]:
# From https://docs.python.org/3/library/itertools.html#itertools-recipes

def powerset(iterable):
   
    "list(powerset([1,2,3])) --> [(), (1,), (2,), (3,), (1,2), (1,3), (2,3), (1,2,3)]"
    s = list(iterable)  # Convert iterable to list
    # Return chained combinations of all lengths (including empty set)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

def optimal_set_cover(universe, subsets, costs):
    """
    Optimal algorithm for the set cover problem using brute force.
    It finds the minimum cost subcollection of subsets that covers all elements of the universe.

    Args:
        universe (list): Universe of elements.
        subsets (dict): Subsets of U {S1:elements, S2:elements}.
        costs (dict): Costs of each subset in S {S1:cost, S2:cost...}.

    Returns:
        The minimum cost set cover.
    """
    pset = powerset(subsets.keys())  # Get all possible combinations (powerset) of subsets
    best_set = None  # To store the best set cover
    best_cost = float("inf")  # Initialize the best cost as infinity

    # Iterate through each combination in the powerset
    for subset in pset:
        covered = set()  # To track elements covered by the current combination
        cost = 0  # To track total cost of the current subset combination

        # Add all elements of the current subset to covered, and accumulate the cost
        for s in subset:
            covered.update(subsets[s])  # Update covered elements with current subset
            cost += costs[s]  # Add the cost of the current subset

        # Check if this combination covers the whole universe and has a lower cost
        if len(covered) == len(universe) and cost < best_cost:
            best_set = subset  # Update best set cover
            best_cost = cost  # Update best cost

    return best_set  # Return the best set cover

def greedy_set_cover(universe, subsets, costs):
    """
    Greedy approximation algorithm for the set cover problem.
    It selects subsets based on the minimum cost-to-new-elements ratio.

    Args:
        universe (list): Universe of elements.
        subsets (dict): Subsets of U {S1:elements, S2:elements}.
        costs (dict): Costs of each subset in S {S1:cost, S2:cost...}.

    Returns:
        An approximate set cover with reasonably good performance.
    """
    # Flatten all elements in subsets and check if they cover the universe
    elements = set(e for s in subsets.keys() for e in subsets[s])
    # If the elements don't cover the entire universe, the input is invalid
    if elements != universe:
        return None

    covered = set()  # To track the elements covered by selected subsets
    cover_sets = []  # To store the subsets selected for the set cover

    # Continue until all elements of the universe are covered
    while covered != universe:
        min_cost_elem_ratio = float("inf")  # Initialize the minimum cost-to-element ratio
        min_set = None  # Initialize the best subset for the next step

        # Find the subset with the minimum cost-to-new-elements ratio
        for s, elements in subsets.items():
            new_elements = len(elements - covered)  # Calculate uncovered elements in this subset
            if new_elements != 0:  # Avoid division by zero
                cost_elem_ratio = costs[s] / new_elements  # Compute the ratio
                if cost_elem_ratio < min_cost_elem_ratio:  # Update if a better ratio is found
                    min_cost_elem_ratio = cost_elem_ratio
                    min_set = s

        cover_sets.append(min_set)  # Add the best subset to the cover
        covered |= subsets[min_set]  # Update covered elements with this subset

    return cover_sets  # Return the selected cover sets

if __name__ == '__main__':
    universe = {1, 2, 3, 4, 5}  # Define universe
    subsets = {'S1': {4, 1, 3}, 'S2': {2, 5}, 'S3': {1, 4, 3, 2}}  # Define subsets
    costs = {'S1': 5, 'S2': 10, 'S3': 3}  # Define costs for each subset

    # Find optimal cover and its cost using the brute force approach
    optimal_cover = optimal_set_cover(universe, subsets, costs)
    optimal_cost = sum(costs[s] for s in optimal_cover)  # Calculate total cost of optimal cover

    # Find greedy cover and its cost using the greedy approach
    greedy_cover = greedy_set_cover(universe, subsets, costs)
    greedy_cost = sum(costs[s] for s in greedy_cover)  # Calculate total cost of greedy cover

    # Print results
    print('Optimal Set Cover:')
    print(optimal_cover)
    print('Cost = %s' % optimal_cost)

    print('Greedy Set Cover:')
    print(greedy_cover)
    print('Cost = %s' % greedy_cost)
